In [ ]:
# external libraries
import numpy as np
# local libraries
from nn import NeuralNetwork, LinearLayer, ActivationFunction, LossFunction

class GridSearch():
    def __init__(self, trainer)

    def k_fold():
        estimator.train()
        print("hello")

def nested_k_fold(x, y, n_folds, model: NeuralNetwork):
    print("hello")

In [ ]:
net = NeuralNetwork([
    LinearLayer((8, 16)),
    ActivationFunction(),
    LinearLayer((16, 16)),
    ActivationFunction(),
    LinearLayer((16, 2))
])


In [ ]:
holdout_validation(net)